# Identifying the most-mentioned countries in the 2021 US spending omnibus
## (A sloppy attempt by someone who is a mostly frontend engineer who hasn't worked with Python in awhile and doesn't know enough about NLP)

## 1. Read bill text

To get the text file used here, I extracted the bill's plaintext from this [source PDF](https://docs.house.gov/billsthisweek/20201221/BILLS-116HR133SA-RCP-116-68.pdf) via xpdf's `pdftotext`. Unfortunately, this leaves the line numbering in place, and there are hyphenated word breaks that cross lines. I cleaned up a bit to unwrap words that got wrapped with a hyphen across multiple lines by removing all matches of this regex:

```
-$\n+\d+\s+
```

The text version is still imperfect after that, with some multiword country names potentially split across multiple lines, possibly causing some country references to be broken, but I don't expect this to affect the count too much.

In [24]:
bill_text = open('public-law-116-94.txt', 'r', encoding="ISO-8859-1").read()
print(len(bill_text))

2483930


## 2. Use `flashgeotext` to get (rough) country counts

This bill is very big, almost 7 million characters. I tried working with [Spacy](https://spacy.io/) but had to break up the work into chunks so as not to hit memory limits, and ultimately didn't get a great result for various other reasons, mostly patience (Spacy's stock models aren't trained to extract *just* countries, instead extracting "geopolitical entities", and I got impatient with its runtime while trying to figure out how to appropriately filter just to countries).

[flashgeotext](https://github.com/iwpnd/flashgeotext) is simpler and looks more like what I wanted: just produce a count of mentions of countries with no intermediate steps required, and a much faster search method.

In [25]:
from flashgeotext.geotext import GeoText

In [26]:
geotext = GeoText()

2020-12-22 23:31:17.438 | DEBUG    | flashgeotext.lookup:add:194 - cities added to pool
2020-12-22 23:31:17.553 | DEBUG    | flashgeotext.lookup:add:194 - countries added to pool
2020-12-22 23:31:17.554 | DEBUG    | flashgeotext.lookup:_add_demo_data:225 - demo data loaded for: ['cities', 'countries']


In [27]:
countries = geotext.extract(bill_text, span_info=False).get('countries')

In [28]:
print(countries)

{'United States': {'count': 1112}, 'Mexico': {'count': 11}, 'Puerto Rico': {'count': 52}, 'China': {'count': 62}, 'Russia': {'count': 33}, 'Palau': {'count': 1}, 'Iran': {'count': 15}, 'North Korea': {'count': 11}, 'South Korea': {'count': 1}, 'Japan': {'count': 2}, 'Cuba': {'count': 15}, 'Afghanistan': {'count': 23}, 'Taiwan': {'count': 2}, 'Canada': {'count': 8}, 'United Kingdom': {'count': 2}, 'Israel': {'count': 49}, 'Bahrain': {'count': 1}, 'Myanmar': {'count': 18}, 'Cambodia': {'count': 12}, 'Colombia': {'count': 15}, 'Egypt': {'count': 16}, 'Ethiopia': {'count': 1}, 'Iraq': {'count': 17}, 'Lebanon': {'count': 8}, 'Pakistan': {'count': 6}, 'Philippines': {'count': 3}, 'Sudan': {'count': 13}, 'Sri Lanka': {'count': 5}, 'Zimbabwe': {'count': 4}, 'Jordan': {'count': 6}, 'Tunisia': {'count': 4}, 'Ukraine': {'count': 12}, 'Palestine': {'count': 13}, 'Morocco': {'count': 2}, 'Saudi Arabia': {'count': 5}, 'Cameroon': {'count': 3}, 'Central African Republic': {'count': 2}, 'Democratic Re

## 3. Spot-check

Spot-checking the above just by searching inside the original PDFs, the results look decent. But there are some issues that would require a fancier approach to fix: the "Sudan" count includes South Sudan mentions (outdated country list?), and "Mexico" includes New Mexico, which bloats the count for mentions of Mexico-the-country to ~25% higher than it should be. The model isn't aware of US states so it's failing to eliminate New Mexico as a longer string match. So we need to go at least a bit fancier to get a good result.

## 4. Try wiring up `pycountry`'s country database + `pyahocorasick`

flashgeotext uses an [Aho–Corasick algorithm](https://en.wikipedia.org/wiki/Aho%E2%80%93Corasick_algorithm) implementation under the hood for its string searching. Let's drop down one level of abstraction and re-implement to handle the country lookup we need by building an "automaton" more directly, using [`pyahocorasick`](https://pypi.org/project/pyahocorasick/). Reading around, it looks like it might have better handling for overlapping matches than the implementation used in flashgeotext.

We basically do this by dumping [`pycountry`](https://pypi.org/project/pycountry/)'s database of countries and US subdivisions/states into ahocorasick:

In [29]:
import ahocorasick
import pycountry

A = ahocorasick.Automaton()

# Add countries to the automaton
for country in pycountry.countries:    
    # `country.name` is the [ISO English "short country name"](https://unstats.un.org/unsd/tradekb/knowledgebase/country-code).
    # Unfortunately, it's not always all that short. We need to special-case to handle some countries with long "short" names
    # that are commonly abbreviated further:
    shorter_names = {
        'Russian Federation': 'Russia',
        'Korea, Republic of': 'South Korea',
        "Korea, Democratic People's Republic of": 'North Korea'
    }

    shorter_name = None
    
    if country.name in shorter_names:
        shorter_name = shorter_names[country.name]
    elif "," in country.name:
        # Catch-all for other long short names
        # e.g. "Iran, Islamic Republic of" => "Iran"; "Bolivia, Plurinational State of" => "Bolivia"
        # FIXME: Should manually list these out instead in shorter_names because there are edge cases here like
        # "Saint Helena, Ascension and Tristan da Cunha"
        shorter_name = country.name.split(",")[0]
    
    # There are some political implications to choosing ISO English short name representation
    # or more common vernacular. For example, Taiwan is "Taiwan, Province of China" per its
    # ISO short name. Here let's opt for the most commonly-used naming, which also
    # lines up with most of the naming in the bill text in this case.
    entity = ('COUNTRY', shorter_name or country.name,)
    
    A.add_word(country.name, entity)
    if shorter_name is not None:
        A.add_word(shorter_name, entity)    
    if hasattr(country, 'official_name'):
        A.add_word(country.official_name, entity)

# Add US states and territories
for subdivision in pycountry.subdivisions.get(country_code='US'):
    if subdivision.type == 'State':
        A.add_word(subdivision.name, ('US_STATE', subdivision.name,))
    else:
        A.add_word(subdivision.name, ('US_TERRITORY', subdivision.name,))
        
A.make_automaton()

## Iterate through matches the automaton found; get counts

In [30]:
import re

last_idx = None
entities = []

word_char = re.compile('\w')

for (idx, entity,) in A.iter(bill_text):
    # Look ahead to next character. If it's a word character, we don't want this
    # entity - it's a prefix match for something else (e.g. "India" in "Indian")
    next_char = bill_text[idx + 1 : idx + 2]
    if (word_char.match(next_char)):
        continue

    if idx == last_idx:
        # Overlapping match found. Filter to the longest match; only replace previous match
        # if new entity name is longer.
        # (e.g. when "New Mexico" match is followed by "Mexico" - "New Mexico" wins)
        if len(entity[1]) > len(entities[-1][1]):
            entities[-1] = entity
    else:
        entities.append(entity)
    last_idx = idx

countries = [entity for entity in entities if entity[0] == 'COUNTRY']
country_counts = {}
for (_, country) in countries:
    if country not in country_counts:
        country_counts[country] = 1
    else:
        country_counts[country] += 1
        
country_counts_sorted = sorted(country_counts.items(), key=lambda kv: kv[1], reverse=True)
print('\n'.join([k[0] + ', ' + str(k[1]) for k in country_counts_sorted]))

United States, 1062
Venezuela, 90
China, 56
Israel, 49
Cyprus, 47
Russia, 33
Greece, 29
Afghanistan, 23
Iraq, 17
Egypt, 16
Iran, 15
Cuba, 15
Colombia, 15
Palestine, 13
Cambodia, 12
Mexico, 11
North Korea, 11
Ukraine, 11
Hong Kong, 10
Canada, 8
Haiti, 8
Lebanon, 8
Sudan, 8
Somalia, 6
Honduras, 6
Pakistan, 6
Jordan, 6
Nepal, 6
Guatemala, 5
South Sudan, 5
Sri Lanka, 5
Yemen, 5
Saudi Arabia, 5
Libya, 4
Nicaragua, 4
Zimbabwe, 4
Tunisia, 4
Costa Rica, 4
Micronesia, 3
El Salvador, 3
Philippines, 3
Cameroon, 3
Congo, 3
Bangladesh, 3
Panama, 3
Turkey, 3
Iceland, 3
Norway, 3
Marshall Islands, 2
Samoa, 2
Northern Mariana Islands, 2
Japan, 2
Taiwan, 2
Morocco, 2
Central African Republic, 2
Chad, 2
Malawi, 2
Belize, 2
Peru, 2
Ecuador, 2
Chile, 2
Guinea, 2
Palau, 1
Bahrain, 1
Ethiopia, 1
Uzbekistan, 1
Western Sahara, 1
Niger, 1
Nigeria, 1
Thailand, 1
India, 1
Trinidad and Tobago, 1
Bolivia, 1
France, 1
Germany, 1
Italy, 1
Spain, 1
Portugal, 1
Sweden, 1
Netherlands, 1
United Kingdom, 1
Uruguay, 1
Arg

## Summary

This is where I stopped due to time constraints. It seems to be much more accurate than the attempt earlier in this notebook. Spot-checking it shows we seem to be matching genuine references to countries, not nationalities, and we're not confusing "New Mexico" with "Mexico". But:

 - "Gulf of Mexico" counts as a "Mexico" reference. To handle that, we need some way of disambiguating non-country, non-US-state geopolitical entities like "Gulf of Mexico". That would involve either semantic awareness of "Gulf of Mexico" as a concept, or syntactic awareness (Gulf of Mexico == proper noun that should trump the nested Mexico proper noun).
 - ~Much worse, "North Korea" is nowhere to be seen - I sloppily stripped the ", Republic of" and ", Democratic People's Republic of" suffixes from the ISO short name! This carelessly reunifies Korea.~
 
For my purposes (practice exercise), I think I'll be happy with this after adding a new list of short country names that patches the ISO short names. This should fix the last issue, but not the first one. To get more accuracy, a fancier model with more semantic awareness is likely needed.